**Brainstorming**:
- comparison (soda generator)
- arithmetic
- bitshifts
- Note on semantics
- Exercises/examples: grammar, dice arithmetic, graph and symbolic computation, shortcuts
- formating?



Operator overloading
====================

**Outline**:
1. First example
2. Comparison overloading
3. Arithmetic overloading
4. Shift operators
5. Other operators
6. Best practices
7. Closing words

## 1. First example

The example below illustrates two operation overloads.

In [13]:
from __future__ import annotations

from typing import Union
import random


class Gram:
    def __or__(self, other: Union[Gram, str]) -> OrRule:
        return OrRule(self, other)
    
    def __add__(self, other: Union[Gram, str]) -> CFRule:
        return CFRule(self, other)

class Rule(Gram):
    def __init__(self, *operands: Union[Gram, str]) -> None:
        super().__init__()
        def cast(x: Union[Gram, str]) -> Gram:
            if isinstance(x, str):
                x = Terminal(x)
            return x
        
        self._operands = tuple(cast(x) for x in operands)

class OrRule(Rule):
    def __str__(self) -> str:
        return str(random.choice(self._operands))
    
class CFRule(Rule):

    def __str__(self) -> str:
        return " ".join(str(x) for x in self._operands)


class Terminal(Gram):
    def __init__(self, symbol: str) -> None:
        self._symbol = symbol

    def __str__(self) -> str:
        return self._symbol
    
    
protagonist = (
    Terminal("Harry Potter") | "Luke Skywalker" | "Frodo Baggins"
)


actions = Terminal("must find") | "must destroy"

macguffin = (
    Terminal("the horcruxes") | "the death star" | "the one true ring"
)

victory = Terminal("to defeat") | "to beat" | "to vanquished" | "to rid the world of"

enemy = Terminal("Voldemort") | "the empire" | "Sauron"


story = protagonist + actions + macguffin + victory + enemy

for _ in range(5):
    print(story)


Harry Potter must find the one true ring to rid the world of the empire
Frodo Baggins must destroy the one true ring to beat Sauron
Luke Skywalker must find the death star to rid the world of Sauron
Frodo Baggins must find the horcruxes to beat the empire
Frodo Baggins must destroy the death star to rid the world of the empire


Overloading `|` and `+` does not allow us to do anything more, so

:question: Why use operation overloading?

:question: What should you be careful with when dealing with operation overloading?

:question: What are good use cases for operation overloading?

:skull: The example above is about context-free grammar. The formal notion of grammar is used extensively in CS from computation theory to compilers, passing through procedural generation and natural language processing.

## 2. Comparison overloading

You can overload the following comparison operators in Python:

| Operator         | Symbol | Dunder   |
|------------------|--------|----------|
| equal            | ==     | `__eq__` |
| not equal        | !=     | `__ne__` |
| lower than       | <      | `__lt__` |
| lower or equal   | <=     | `__le__` |
| greater than     | >      | `__lt__` |
| greater or equal | <      | `__lt__` |

All those relationships are binary, so the methods always take another element as input.

Related to comparison, you can also overwrite the `__bool__` type-cast method of a object so that it can be used as a boolean.


In [4]:
from abc import ABCMeta, abstractmethod
from typing import Any

import pandas as pd


class Test(metaclass=ABCMeta):
    def __init__(self, column_name: str, reference: float) -> None:
        self._column_name = column_name
        self._reference = reference

    @abstractmethod
    def __call__(self, df: pd.DataFrame) -> "TestResult":
        raise NotImplementedError()
    
    def __repr__(self) -> str:
        return f"{self.__class__.__qualname__}({self._column_name!r}, {self._reference!r})"
    
class TestResult:
    def __init__(self, test: Test, result: bool) -> None:
        self._test = test
        self._result = result

    def __repr__(self) -> str:
        return f"{self.__class__.__qualname__}({self._test!r}, {self._result!r})"

    def __bool__(self) -> bool:
        return self._result
    
class LeTest(Test):
    def __call__(self, df: pd.DataFrame) -> TestResult:
        return all(df[self._column_name] <= self._reference)

class EqTest(Test):
    def __call__(self, df: pd.DataFrame) -> TestResult:
        return all(df[self._column_name] == self._reference)
    
class PlaceHolder:
    def __init__(self, column_name: str) -> None:
        self._column_name = column_name

    def __eq__(self, other: float) -> EqTest:
        return EqTest(self._column_name, other)

    def __le__(self, other: float) -> LeTest:
        return LeTest(self._column_name, other)


df = pd.DataFrame({"col": [1, 2, 3, 4]})

test1 = PlaceHolder("col") <= 5
test2 = PlaceHolder("col") == 2

if test1(df):
    print("This should be false:", bool(test2(df)))


This should be false: False


## 3. Arithmetic overloading

### Exposition

You can overload the following comparison operators in Python:

| Operator         | Symbol | Dunder         |
|------------------|--------|----------------|
| add              | +      | `__add__`      |
| subtract         | -      | `__sub__`      |
| multiply         | *      | `__mul__`      |
| divide           | /      | `__truediv__`  |
| divide (integer) | //     | `__floordiv__` |
| power            | **     | `__pow__`      |
| remainder        | %      | `__mod__`      |
| matrix mult.     | @      | `__matmul__`   |
| and              | &      | `__and__`      |
| or               | \|     | `__or__`       |
| xor              | ^      | `__xor__`      |

There are two variants around the operator: the `r`-operators and the `i`-operators. 

In [11]:
class MyFloat:
    def __init__(self, x):
        self.x = x

    def __add__(self, other):
        return self.x + other

class MyRFloat:
    def __init__(self, x):
        self.x = x

    def __add__(self, other):
        return self.x + other

    def __radd__(self, other):
        return other + self.x


print("Case 1:", MyRFloat(2) + 3)
print("Case 2:", 3 + MyRFloat(2))
print("Case 3:", MyFloat(2) + 3)
print("Case 4:", 3 + MyFloat(2))

Case 1: 5
Case 2: 5
Case 3: 5


TypeError: unsupported operand type(s) for +: 'int' and 'MyFloat'

The `i`-variant is about inplace operators. Note that the signature of the function is different:

In [16]:
from __future__ import annotations

class MyNewFloat:
    def __init__(self, x):
        self.x = x

    def __add__(self, other: float) -> float:
        return self.x + other

    def __iadd__(self, other: float) -> MyNewFloat:
        self.x += other
        return self

mnf = MyNewFloat(2)
mnf += 3
mnf.x

5

> :skull: the `operator` module from the standard Python library gives access to shortcuts to invoke the operators.

### Exercise

:hourglass:

Implement a dice class. A dice has $k$ sides and rolling each side has a probability of $\frac{1}{k}$. Using `int(d)` where `d` is a dice should return the result of a dice roll. A dice pool is a collection of dice; using `int(p)` where `p` is a pool should return the sum of the dice rolls.

## 4. Shift operators

You can overload the following bit shift operators in Python (+ the `r` and `i` variants):


| Operator     | Symbol | Dunder         |
|--------------|--------|----------------|
| right shift  | >>     | `__rshift__`   |
| left shift   | <<     | `__lshift__`   |

Technically, the bit shift operators are used to manipulate the bits in binary data. When they are overloaded, it is usually to take advantage of the directionality of the symbols. 

It tends to be used in the following cases:
- directed graph (eg. Airflow);
- move files around.


## 5. Unary operators

You can overload the following unary operators in Python:

| Operator  | Symbol | Dunder       |
|-----------|--------|--------------|
| negative  | -      | `__neg__`    |
| positive  | +      | `__pos__`    |
| invertion | ~      | `__invert__` | 

Here is an example:

## 6. Best practices

## 7. Closing words

See https://docs.python.org/3/library/operator.html for more.

In [ ]:
## 